# Does Specializing More than Once Improve Learning

In [1]:
from res_specialization import *
from matplotlib import pyplot as plt
import sys
import random
import pickle
plt.rcParams['figure.figsize'] = [10, 5]

In [2]:
TRIALS      = 1000
NUM_TO_SPEC = 3
TOL = 5

diff_eq_params = {"x0": [-20, 10, -.5], 
                  "begin": 0, 
                  "end": 60, 
                  "timesteps":60000}

res_params = {"res_sz": 30, 
              "activ_f": np.tanh,
              "connect_p": .12, 
              "ridge_alpha": .00001, 
              "spect_rad": .9, 
              "gamma": 1., 
              "sigma": 0.12,
              "uniform_weights": True
             }

results = dict()


def save_results():
    pickle.dump(results, open("spec_tw_fixed.pkl","wb"))
# end

### Model trajectory

In [3]:
t, train_t, u = lorenz_equ(**diff_eq_params )
train_t = t[30000:55000]
test_t  = t[55000:]

### Re-run grant figure trials to get prediction length


In [4]:
random_err       = []
random_pred      = []
specialized_err  = []
specialized_pred = []


spec_sizes       = []
spec_edges       = []

for i in range(TRIALS):
    
    ######################
    # Random graph
    ######################
    
    # Make rc
    rc = ResComp(3,3,**res_params)
    r_0 = rc.state_0
    err = rc.fit(train_t,u)
    
    # Make predicitions
    test_pre = rc.predict(test_t)
    train_pre = rc.predict(train_t, r_0=r_0)
    
    # Store Error
    random_err.append(err)
    random_pred.append(how_long_accurate(u(test_t), test_pre, tol=TOL))
    
    #######################
    # Specialize graph
    ######################
    
    A = specialize_best_nodes(rc, NUM_TO_SPEC, u, train_t, r_0=r_0)
    spec_sizes.append(A.shape[0])
    spec_edges.append(np.sum(A != 0))
    
    rc = make_res_comp(A, res_params)
    r_0 = rc.state_0
    err = rc.fit(train_t,u)
    test_pre = rc.predict(test_t)
    train_pre = rc.predict(train_t, r_0=r_0)
    specialized_err.append(err)
    specialized_pred.append(how_long_accurate(u(test_t), test_pre, tol=TOL))


In [5]:
results["random_err"]       = random_err
results["random_pred"]      = random_pred
results["specialized_err"]  = specialized_err
results["specialized_pred"] = specialized_pred
results["spec_sizes"]       = spec_sizes
results["spec_edges"]       = spec_edges
save_results()
print("Mean Prediction Length: {}".format(np.mean(random_pred)))
print("Mean Prediction Length: {}".format(np.mean(specialized_pred)))

Mean Prediction Length: 401.4
Mean Prediction Length: 907.203


### Control: Same sizes and same edge densities as previous experiment

In [6]:
match_edge_err = []
match_edge_pred = []

for s,e in zip(spec_sizes,spec_edges):
    # Test random graph with same number of edges as specialized graphs
    
    # Make and fit RC
    res_params["res_sz"] = s
    res_params["connect_p"] = e/(s**2)
    rc = ResComp(3,3,**res_params)
    r_0 = rc.state_0
    err = rc.fit(train_t,u)
    
    # Predict
    test_pre = rc.predict(test_t)
    train_pre = rc.predict(train_t, r_0=r_0)
    match_edge_err.append(err)
    match_edge_pred.append(how_long_accurate(u(test_t), test_pre, tol=TOL))


In [7]:
results["match_edge_err"] = match_edge_err
results["match_edge_pred"] = match_edge_pred
print("Mean Prediction Length: {}".format(np.mean(match_edge_pred)))
save_results()


Mean Prediction Length: 743.825


### Specialization without finding best nodes

In [8]:
res_params = {"res_sz": 30, 
              "activ_f": np.tanh,
              "connect_p": .12, 
              "ridge_alpha": .00001, 
              "spect_rad": .9, 
              "gamma": 1., 
              "sigma": 0.12,
              "uniform_weights": True
             }

random_specialized_err  = []
random_specialized_pred = []
rspec_sizes         = []
rspec_edges         = []

for i in range(TRIALS):
    # Random graph
    rc = ResComp(3,3,**res_params)
    
    # Turn reservoir to integer adj matrix
    A = rc.res
    for j in range(A.shape[0]): A[j,j] = 0
    A = (A != 0)*1
    
    # Specialize the reservoir
    A = specializeGraph(A, random.sample(list(range(30)),27))
    
    # Store data
    rspec_sizes.append(A.shape[0])
    rspec_edges.append(np.sum(A != 0))
    
    # Make new reservoir
    rc = make_res_comp(A, res_params)
    r_0 = rc.state_0
    err = rc.fit(train_t,u)
    
    # Get predicitons
    test_pre = rc.predict(test_t)
    train_pre = rc.predict(train_t, r_0=r_0)
    
    # Store data
    random_specialized_err.append(err)
    random_specialized_pred.append(how_long_accurate(u(test_t), test_pre, tol=TOL))


In [9]:
results["random_specialized_err"]  = random_specialized_err
results["random_specialized_pred"] = random_specialized_pred
results["rspec_sizes"]             = rspec_sizes
results["rspec_edges"]             = rspec_edges

print("Mean Prediction Length: {}".format(np.mean(random_specialized_pred)))
save_results()


Mean Prediction Length: 811.89


### Control: Same sizes and same edge densities as previous experiment


In [ ]:
rand_match_edge_err  = []
rand_match_edge_pred = []

for s,e in zip(rspec_sizes, rspec_edges):
    # Test random graph
    
    # Make and fit RC
    res_params["res_sz"] = s
    res_params["connect_p"] = e/(s**2)
    rc = ResComp(3,3,**res_params)
    r_0 = rc.state_0
    err = rc.fit(train_t,u)
    
    # Predict
    test_pre = rc.predict(test_t)
    train_pre = rc.predict(train_t, r_0=r_0)
    rand_match_edge_err.append(err)
    rand_match_edge_pred.append(how_long_accurate(u(test_t), test_pre, tol=TOL))
    

In [ ]:
results["rand_match_edge_err"]  = rand_match_edge_err
results["rand_match_edge_pred"] = rand_match_edge_pred
print("Mean Prediction Length: {}".format(np.mean(rand_match_edge_pred)))
save_results()


Mean Prediction Length: 682.384


### Targeted Specialize Twice

In [ ]:
res_params = {"res_sz": 30, 
              "activ_f": np.tanh,
              "connect_p": .12, 
              "ridge_alpha": .00001, 
              "spect_rad": .9, 
              "gamma": 1., 
              "sigma": 0.12,
              "uniform_weights": True
             }

spec_tw_pre = []
spec_tw_err = []
tw_edges = []
tw_sizes = []
HOW_MANY_SPEC = 2

for i in range(TRIALS):
    # Make rc
    rc = ResComp(3,3,**res_params)
    r_0 = rc.state_0
    rc.fit(train_t,u)
    
    # Specialize 2 times
    for i in range(HOW_MANY_SPEC):
        A = specialize_best_nodes(rc, NUM_TO_SPEC, u, train_t, r_0=r_0)
        rc = make_res_comp(A, res_params)
        r_0 = rc.state_0
        err = rc.fit(train_t,u)
    
    # Predict the system states
    tw_sizes.append(A.shape[0])
    tw_edges.append(np.sum(A != 0))
    test_pre = rc.predict(test_t)
    train_pre = rc.predict(train_t, r_0=r_0)
    spec_tw_err.append(err)
    spec_tw_pre.append(how_long_accurate(u(test_t), test_pre,tol=TOL))

In [ ]:
results["spec_tw_pre"] = spec_tw_pre
results["spec_tw_err"] = spec_tw_err
results["tw_edges"] = tw_edges
results["tw_sizes"] = tw_sizes
save_results()


In [ ]:
print("Mean Prediction Length: {}".format(np.mean(spec_tw_pre)))


Mean Prediction Length: 967.276


### Control: Same sizes and same edge densities as previous experiment

In [ ]:
tw_match_edge_err  = []
tw_match_edge_pred = []

for s,e in zip(tw_sizes, tw_edges):
    # Test random graph
    
    # Make and fit RC
    res_params["res_sz"] = s
    res_params["connect_p"] = e/(s**2)
    rc = ResComp(3,3,**res_params)
    r_0 = rc.state_0
    err = rc.fit(train_t,u)
    
    # Predict
    test_pre = rc.predict(test_t)
    train_pre = rc.predict(train_t, r_0=r_0)
    tw_match_edge_err.append(err)
    tw_match_edge_pred.append(how_long_accurate(u(test_t),test_pre,tol=TOL))
    

In [ ]:
results["tw_match_edge_err"] = tw_match_edge_err
results["tw_match_edge_pred"] = tw_match_edge_pred
save_results()
print("Mean Prediction Length: {}".format(np.mean(tw_match_edge_pred)))


Mean Prediction Length: 778.591


### Targeted Specialize Three Times

In [ ]:
res_params = {"res_sz": 30, 
              "activ_f": np.tanh,
              "connect_p": .12, 
              "ridge_alpha": .00001, 
              "spect_rad": .9, 
              "gamma": 1., 
              "sigma": 0.12,
              "uniform_weights": True
             }

spec_thr_pre = []
spec_thr_err = []
thr_edges = []
thr_sizes = []
HOW_MANY_SPEC = 3

for i in range(TRIALS):
    # Make RC
    rc = ResComp(3,3,**res_params)
    r_0 = rc.state_0
    rc.fit(train_t,u)
    
    # Specialize 3 times
    for i in range(HOW_MANY_SPEC):
        A = specialize_best_nodes(rc, NUM_TO_SPEC, u, train_t, r_0=r_0)
        rc = make_res_comp(A, res_params)
        r_0 = rc.state_0
        err = rc.fit(train_t,u)
    
    # Make predicitons
    thr_sizes.append(A.shape[0])
    thr_edges.append(np.sum(A != 0))
    test_pre = rc.predict(test_t)
    train_pre = rc.predict(train_t, r_0=r_0)
    spec_thr_err.append(err)
    spec_thr_pre.append(how_long_accurate(u(test_t), test_pre,tol=TOL))


In [ ]:
results["spec_thr_pre"] = spec_thr_pre
results["spec_thr_err"] = spec_thr_err
results["thr_edges"] = thr_edges
results["thr_sizes"] = thr_sizes
print("Mean Prediction Length: {}".format(np.mean(spec_thr_pre)))

save_results()


Mean Prediction Length: 980.5


### Control: Same sizes and same edge densities as previous experiment

In [ ]:
thr_match_edge_err  = []
thr_match_edge_pred = []

for s,e in zip(thr_sizes, thr_edges):
    # Test random graph
    
    # Make and fit RC
    res_params["res_sz"] = s
    res_params["connect_p"] = e/(s**2)
    rc = ResComp(3,3,**res_params)
    r_0 = rc.state_0
    err = rc.fit(train_t,u)
    
    # Predict systems
    test_pre = rc.predict(test_t)
    train_pre = rc.predict(train_t, r_0=r_0)
    thr_match_edge_err.append(err)
    thr_match_edge_pred.append(how_long_accurate(u(test_t),test_pre,tol=TOL))
    

In [ ]:
results["thr_match_edge_err"] = thr_match_edge_err
results["thr_match_edge_pred"] = thr_match_edge_pred
print("Mean Prediction Length: {}".format(np.mean(thr_match_edge_pred)))

save_results()


Mean Prediction Length: 793.779


## Organize results into dataframes


In [ ]:
import pandas as pd

In [ ]:
labels = ["random_specialized","specialized","spec_twice", "spec_three"]
df = pd.DataFrame(results)
df.describe()

,random_err,random_pred,specialized_err,specialized_pred,spec_sizes,spec_edges,match_edge_err,match_edge_pred,random_specialized_err,random_specialized_pred,...,tw_edges,tw_sizes,tw_match_edge_err,tw_match_edge_pred,spec_thr_pre,spec_thr_err,thr_edges,thr_sizes,thr_match_edge_err,thr_match_edge_pred
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,1.995859,401.400000,0.091755,907.203000,117.148000,239.005000,0.195689,743.825000,0.184244,811.89000,...,284.656000,145.585000,0.128446,778.591000,980.500000,0.044785,359.281000,188.122000,0.078432,793.779000
std,0.916472,268.433024,0.087752,425.474371,75.549625,130.530223,0.255036,244.446797,0.165214,242.22574,...,153.039128,91.486505,0.223234,200.876452,524.618937,0.032344,328.894843,179.471165,0.132447,218.153673
min,0.719004,0.000000,0.016666,501.000000,29.000000,82.000000,0.008289,11.000000,0.017501,271.00000,...,89.000000,40.000000,0.007046,35.000000,616.000000,0.010537,99.000000,49.000000,0.004964,0.000000
25%,1.458024,132.750000,0.039568,779.500000,72.000000,166.000000,0.050122,655.750000,0.070871,705.00000,...,191.000000,89.000000,0.029214,685.000000,841.000000,0.025931,219.750000,109.000000,0.018298,703.000000
50%,1.789611,466.500000,0.063113,858.000000,95.000000,204.000000,0.116187,740.500000,0.127277,799.00000,...,246.500000,122.000000,0.060508,766.000000,903.000000,0.035312,290.000000,150.000000,0.037511,779.500000
75%,2.285541,613.250000,0.110930,936.000000,132.000000,263.250000,0.229266,832.250000,0.240808,892.00000,...,323.500000,169.000000,0.143350,857.000000,984.250000,0.050492,423.000000,227.000000,0.079134,869.250000
max,9.091017,1128.000000,0.947470,5000.000000,654.000000,1247.000000,2.240181,5000.000000,1.299541,4779.00000,...,1311.000000,842.000000,3.178008,4292.000000,5000.000000,0.337257,8694.000000,4636.000000,1.501932,4334.000000


### Error

In [ ]:
df[["random_specialized_err","specialized_err","spec_tw_err", "spec_thr_err"]].describe()

,random_specialized_err,specialized_err,spec_tw_err,spec_thr_err
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.184244,0.091755,0.065689,0.044785
std,0.165214,0.087752,0.066160,0.032344
min,0.017501,0.016666,0.015098,0.010537
25%,0.070871,0.039568,0.031122,0.025931
50%,0.127277,0.063113,0.043814,0.035312
75%,0.240808,0.110930,0.071923,0.050492
max,1.299541,0.947470,0.656936,0.337257


### Error Controls

In [ ]:
df[["rand_match_edge_err", "match_edge_err", "tw_match_edge_err", "thr_match_edge_err"]].describe()

,rand_match_edge_err,match_edge_err,tw_match_edge_err,thr_match_edge_err
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.408749,0.195689,0.128446,0.078432
std,0.499109,0.255036,0.223234,0.132447
min,0.008140,0.008289,0.007046,0.004964
25%,0.131431,0.050122,0.029214,0.018298
50%,0.260950,0.116187,0.060508,0.037511
75%,0.489982,0.229266,0.143350,0.079134
max,4.822411,2.240181,3.178008,1.501932


### Prediction

In [ ]:
df[["random_specialized_pred","specialized_pred","spec_tw_pre", "spec_thr_pre"]].describe()

,random_specialized_pred,specialized_pred,spec_tw_pre,spec_thr_pre
count,1000.00000,1000.000000,1000.000000,1000.000000
mean,811.89000,907.203000,967.276000,980.500000
std,242.22574,425.474371,555.022529,524.618937
min,271.00000,501.000000,471.000000,616.000000
25%,705.00000,779.500000,818.750000,841.000000
50%,799.00000,858.000000,890.000000,903.000000
75%,892.00000,936.000000,966.250000,984.250000
max,4779.00000,5000.000000,5000.000000,5000.000000


### Prediction Control

In [ ]:
df[["rand_match_edge_pred", "match_edge_pred", "tw_match_edge_pred", "thr_match_edge_pred"]].describe()

,rand_match_edge_pred,match_edge_pred,tw_match_edge_pred,thr_match_edge_pred
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,682.384000,743.825000,778.591000,793.779000
std,197.919369,244.446797,200.876452,218.153673
min,0.000000,11.000000,35.000000,0.000000
25%,595.000000,655.750000,685.000000,703.000000
50%,701.500000,740.500000,766.000000,779.500000
75%,801.250000,832.250000,857.000000,869.250000
max,1229.000000,5000.000000,4292.000000,4334.000000


### Sizes

In [ ]:
sizes = df[["rspec_sizes", "spec_sizes", "tw_sizes", "thr_sizes"]]
sizes.describe()

,rspec_sizes,spec_sizes,tw_sizes,thr_sizes
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,79.352000,117.148000,145.585000,188.122000
std,45.842094,75.549625,91.486505,179.471165
min,27.000000,29.000000,40.000000,49.000000
25%,54.000000,72.000000,89.000000,109.000000
50%,68.000000,95.000000,122.000000,150.000000
75%,89.000000,132.000000,169.000000,227.000000
max,531.000000,654.000000,842.000000,4636.000000


### Edges

In [ ]:
edges = df[["rspec_edges", "spec_edges", "tw_edges", "thr_edges"]]
edges.describe()

,rspec_edges,spec_edges,tw_edges,thr_edges
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,180.819000,239.005000,284.656000,359.281000
std,79.950124,130.530223,153.039128,328.894843
min,58.000000,82.000000,89.000000,99.000000
25%,136.000000,166.000000,191.000000,219.750000
50%,162.500000,204.000000,246.500000,290.000000
75%,200.000000,263.250000,323.500000,423.000000
max,922.000000,1247.000000,1311.000000,8694.000000


### Connectivity: (num edges)/(num nodes)^2

In [ ]:
labels = ["random_specialized","specialized","spec_twice", "spec_three"]
edge_prob = pd.DataFrame(np.array(edges)/(np.array(sizes)**2),columns=labels)
edge_prob.describe()

,random_specialized,specialized,spec_twice,spec_three
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.037408,0.024197,0.018591,0.014229
std,0.017080,0.012454,0.010021,0.007622
min,0.003270,0.002850,0.001849,0.000405
25%,0.024970,0.015021,0.011294,0.008198
50%,0.035205,0.022263,0.016665,0.012932
75%,0.047535,0.032023,0.024022,0.018522
max,0.096667,0.101070,0.058750,0.048313


In [ ]:
df[["random_err", "random_pred"]].describe()

,random_err,random_pred
count,1000.000000,1000.000000
mean,1.995859,401.400000
std,0.916472,268.433024
min,0.719004,0.000000
25%,1.458024,132.750000
50%,1.789611,466.500000
75%,2.285541,613.250000
max,9.091017,1128.000000


In [32]:
pred = df[[ "random_pred", "random_specialized_pred","specialized_pred","spec_tw_pre", "spec_thr_pre"]]
pred

,random_pred,random_specialized_pred,specialized_pred,spec_tw_pre,spec_thr_pre
0,816,772,1019,842,790
1,206,1217,871,844,889
2,578,711,909,731,783
3,237,819,910,822,823
4,447,828,1091,976,827
...,...,...,...,...,...
995,606,682,761,702,839
996,405,1033,839,786,784
997,615,988,1272,964,883
998,121,640,1035,914,4386
